In [6]:
import pandas as pd
import numpy as np
from sklearn.cross_validation import StratifiedKFold, KFold
import xgboost
from sklearn.grid_search import ParameterGrid
from sklearn.metrics import mean_squared_error

CLASS = False  # Whether classification or regression
SCORE_MIN = True  # Optimizing score through minimum
k = 5  # Number of folds
best_score = 10
best_params = None
best_iter = None

train_name = 'train.csv'
test_name = 'test.csv'
#submission_name = '/input/sample_submission.csv'


# Read files
train = pd.DataFrame.from_csv(train_name)
train = train.fillna(-1)
test = pd.DataFrame.from_csv(test_name)
test = test.fillna(-1)
#submission = pd.DataFrame.from_csv(submission_name)
# Extract target
target = train['SalePrice']
del train['SalePrice']

# Label nominal variables to numbers
columns = train.columns.values
nom_numeric_cols = ['MSSubClass']
dummy_train = []
dummy_test = []
for col in columns:
    # Only works for nominal data without a lot of factors
    if train[col].dtype.name == 'object' or col in nom_numeric_cols:
        dummy_train.append(pd.get_dummies(train[col].values.astype(str), col))
        dummy_train[-1].index = train.index
        dummy_test.append(pd.get_dummies(test[col].values.astype(str), col))
        dummy_test[-1].index = test.index
        del train[col]
        del test[col]
train = pd.concat([train] + dummy_train, axis=1)
test = pd.concat([test] + dummy_test, axis=1)

# Use only common columns
columns = []
for col_a in train.columns.values:
    if col_a in test.columns.values:
        columns.append(col_a)
train = train[columns]
test = test[columns]

# CV
train = np.array(train)
target = np.log(np.array(target))  # Changes to Log
test = np.array(test)
print(train.shape, test.shape)

if CLASS:
    kfold = StratifiedKFold(target, k)
else:
    kfold = KFold(train.shape[0], k)

early_stopping = 50

param_grid = [
              {'silent': [1],
               'nthread': [2],
               'eval_metric': ['rmse'],
               'eta': [0.03],
               'objective': ['reg:linear'],
               'max_depth': [5, 7],
               'num_round': [1000],
               'subsample': [0.2, 0.4, 0.6],
               'colsample_bytree': [0.3, 0.5, 0.7],
               }
              ]



(1460, 299) (1459, 299)


In [12]:
pd.DataFrame(train).head()

,0,1,2,3,4,5,6,7,8,9,...,289,290,291,292,293,294,295,296,297,298
0,65.0,8450.0,7.0,5.0,2003.0,2003.0,196.0,706.0,0.0,150.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,80.0,9600.0,6.0,8.0,1976.0,1976.0,0.0,978.0,0.0,284.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,68.0,11250.0,7.0,5.0,2001.0,2002.0,162.0,486.0,0.0,434.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,60.0,9550.0,7.0,5.0,1915.0,1970.0,0.0,216.0,0.0,540.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
4,84.0,14260.0,8.0,5.0,2000.0,2000.0,350.0,655.0,0.0,490.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


In [13]:

# Hyperparmeter grid optimization
for params in ParameterGrid(param_grid):
    print(params)
    # Determine best n_rounds
    xgboost_rounds = []
    for train_index, test_index in kfold:
        X_train, X_test = train[train_index], train[test_index]
        y_train, y_test = target[train_index], target[test_index]

        xg_train = xgboost.DMatrix(X_train, label=y_train)
        xg_test = xgboost.DMatrix(X_test, label=y_test)

        watchlist = [(xg_train, 'train'), (xg_test, 'test')]

        num_round = params['num_round']
        xgclassifier = xgboost.train(params, xg_train, num_round,
                                     watchlist,
                                     early_stopping_rounds=early_stopping);
        xgboost_rounds.append(xgclassifier.best_iteration)

    num_round = int(np.mean(xgboost_rounds))
    print('The best n_rounds is %d' % num_round)
    # Solve CV
    rmsle_score = []
    for cv_train_index, cv_test_index in kfold:
        X_train, X_test = train[cv_train_index, :], train[cv_test_index, :]
        y_train, y_test = target[cv_train_index], target[cv_test_index]

        # train machine learning
        xg_train = xgboost.DMatrix(X_train, label=y_train)
        xg_test = xgboost.DMatrix(X_test, label=y_test)

        watchlist = [(xg_train, 'train'), (xg_test, 'test')]

        xgclassifier = xgboost.train(params, xg_train, num_round);

        # predict
        predicted_results = xgclassifier.predict(xg_test)
        rmsle_score.append(np.sqrt(mean_squared_error(y_test, predicted_results)))

    if SCORE_MIN:
        if best_score > np.mean(rmsle_score):
            print(np.mean(rmsle_score))
            print('new best')
            best_score = np.mean(rmsle_score)
            best_params = params
            best_iter = num_round
    else:
        if best_score < np.mean(rmsle_score):
            print(np.mean(rmsle_score))
            print('new best')
            best_score = np.mean(rmsle_score)
            best_params = params
            best_iter = num_round

# Solution using best parameters
print('best params: %s' % best_params)
print('best score: %f' % best_score)
xg_train = xgboost.DMatrix(train, label=target)
xg_test = xgboost.DMatrix(test)
watchlist = [(xg_train, 'train')]
num_round = best_iter  # already int
xgclassifier = xgboost.train(best_params, xg_train, num_round, watchlist);


{'silent': 1, 'subsample': 0.2, 'eta': 0.03, 'num_round': 1000, 'colsample_bytree': 0.3, 'eval_metric': 'rmse', 'objective': 'reg:linear', 'nthread': 2, 'max_depth': 5}
[0]	train-rmse:11.1852	test-rmse:11.1979
Multiple eval metrics have been passed: 'test-rmse' will be used for early stopping.

Will train until test-rmse hasn't improved in 50 rounds.
[1]	train-rmse:10.8531	test-rmse:10.8657
[2]	train-rmse:10.5289	test-rmse:10.5416
[3]	train-rmse:10.2136	test-rmse:10.2262
[4]	train-rmse:9.90947	test-rmse:9.92207
[5]	train-rmse:9.6152	test-rmse:9.62778
[6]	train-rmse:9.32729	test-rmse:9.33985
[7]	train-rmse:9.04976	test-rmse:9.0623
[8]	train-rmse:8.78082	test-rmse:8.79334
[9]	train-rmse:8.5197	test-rmse:8.5322
[10]	train-rmse:8.26381	test-rmse:8.27628
[11]	train-rmse:8.01557	test-rmse:8.02802
[12]	train-rmse:7.77655	test-rmse:7.78897
[13]	train-rmse:7.54346	test-rmse:7.55585
[14]	train-rmse:7.31853	test-rmse:7.3309
[15]	train-rmse:7.10094	test-rmse:7.11328
[16]	train-rmse:6.88988	test-rm

[183]	train-rmse:0.12255	test-rmse:0.138231
[184]	train-rmse:0.121506	test-rmse:0.137272
[185]	train-rmse:0.120475	test-rmse:0.136411
[186]	train-rmse:0.119665	test-rmse:0.135712
[187]	train-rmse:0.118929	test-rmse:0.134989
[188]	train-rmse:0.118193	test-rmse:0.134395
[189]	train-rmse:0.117534	test-rmse:0.133603
[190]	train-rmse:0.116855	test-rmse:0.132983
[191]	train-rmse:0.116093	test-rmse:0.132309
[192]	train-rmse:0.115391	test-rmse:0.131549
[193]	train-rmse:0.114575	test-rmse:0.13071
[194]	train-rmse:0.113753	test-rmse:0.130082
[195]	train-rmse:0.113043	test-rmse:0.129387
[196]	train-rmse:0.112622	test-rmse:0.12887
[197]	train-rmse:0.112079	test-rmse:0.128369
[198]	train-rmse:0.111365	test-rmse:0.127622
[199]	train-rmse:0.110958	test-rmse:0.127312
[200]	train-rmse:0.110384	test-rmse:0.12693
[201]	train-rmse:0.109859	test-rmse:0.126545
[202]	train-rmse:0.109287	test-rmse:0.125911
[203]	train-rmse:0.108888	test-rmse:0.125467
[204]	train-rmse:0.108282	test-rmse:0.125121
[205]	train-rm

[366]	train-rmse:0.080344	test-rmse:0.110353
[367]	train-rmse:0.080301	test-rmse:0.110274
[368]	train-rmse:0.080205	test-rmse:0.110359
[369]	train-rmse:0.080138	test-rmse:0.110377
[370]	train-rmse:0.07993	test-rmse:0.110409
[371]	train-rmse:0.079795	test-rmse:0.110421
[372]	train-rmse:0.079681	test-rmse:0.110073
[373]	train-rmse:0.079596	test-rmse:0.110112
[374]	train-rmse:0.079431	test-rmse:0.110025
[375]	train-rmse:0.079305	test-rmse:0.109939
[376]	train-rmse:0.079163	test-rmse:0.10989
[377]	train-rmse:0.079077	test-rmse:0.109969
[378]	train-rmse:0.078901	test-rmse:0.109891
[379]	train-rmse:0.078828	test-rmse:0.109886
[380]	train-rmse:0.078739	test-rmse:0.109803
[381]	train-rmse:0.078611	test-rmse:0.109755
[382]	train-rmse:0.078486	test-rmse:0.109735
[383]	train-rmse:0.078357	test-rmse:0.109712
[384]	train-rmse:0.078276	test-rmse:0.109795
[385]	train-rmse:0.078162	test-rmse:0.109668
[386]	train-rmse:0.078018	test-rmse:0.109677
[387]	train-rmse:0.077935	test-rmse:0.109577
[388]	train-

[34]	train-rmse:4.00377	test-rmse:4.01092
[35]	train-rmse:3.88554	test-rmse:3.89279
[36]	train-rmse:3.76967	test-rmse:3.77679
[37]	train-rmse:3.65772	test-rmse:3.66494
[38]	train-rmse:3.54901	test-rmse:3.55635
[39]	train-rmse:3.44424	test-rmse:3.45162
[40]	train-rmse:3.34251	test-rmse:3.3495
[41]	train-rmse:3.24398	test-rmse:3.25132
[42]	train-rmse:3.14787	test-rmse:3.15527
[43]	train-rmse:3.05535	test-rmse:3.06333
[44]	train-rmse:2.96527	test-rmse:2.97288
[45]	train-rmse:2.87789	test-rmse:2.88523
[46]	train-rmse:2.79285	test-rmse:2.80019
[47]	train-rmse:2.71055	test-rmse:2.71757
[48]	train-rmse:2.63138	test-rmse:2.63835
[49]	train-rmse:2.55486	test-rmse:2.56191
[50]	train-rmse:2.47965	test-rmse:2.48705
[51]	train-rmse:2.40721	test-rmse:2.41449
[52]	train-rmse:2.33615	test-rmse:2.34353
[53]	train-rmse:2.26775	test-rmse:2.27486
[54]	train-rmse:2.20097	test-rmse:2.20808
[55]	train-rmse:2.13586	test-rmse:2.14254
[56]	train-rmse:2.07186	test-rmse:2.07818
[57]	train-rmse:2.01098	test-rmse:2

[221]	train-rmse:0.099081	test-rmse:0.141319
[222]	train-rmse:0.098722	test-rmse:0.141258
[223]	train-rmse:0.098432	test-rmse:0.141229
[224]	train-rmse:0.09822	test-rmse:0.141115
[225]	train-rmse:0.098024	test-rmse:0.141299
[226]	train-rmse:0.09768	test-rmse:0.141187
[227]	train-rmse:0.097383	test-rmse:0.141047
[228]	train-rmse:0.097128	test-rmse:0.140993
[229]	train-rmse:0.096985	test-rmse:0.141003
[230]	train-rmse:0.096693	test-rmse:0.140897
[231]	train-rmse:0.09646	test-rmse:0.140975
[232]	train-rmse:0.096338	test-rmse:0.141289
[233]	train-rmse:0.096133	test-rmse:0.141205
[234]	train-rmse:0.095956	test-rmse:0.141087
[235]	train-rmse:0.09575	test-rmse:0.141195
[236]	train-rmse:0.095502	test-rmse:0.141246
[237]	train-rmse:0.095295	test-rmse:0.141195
[238]	train-rmse:0.094963	test-rmse:0.141248
[239]	train-rmse:0.094641	test-rmse:0.140982
[240]	train-rmse:0.094404	test-rmse:0.140878
[241]	train-rmse:0.094249	test-rmse:0.140981
[242]	train-rmse:0.094076	test-rmse:0.140994
[243]	train-rm

...

[297]	train-rmse:0.087094	test-rmse:0.117091
[298]	train-rmse:0.086966	test-rmse:0.117029
[299]	train-rmse:0.086912	test-rmse:0.117039
[300]	train-rmse:0.086775	test-rmse:0.116913
[301]	train-rmse:0.086588	test-rmse:0.116845
[302]	train-rmse:0.086437	test-rmse:0.116555
[303]	train-rmse:0.086274	test-rmse:0.116613
[304]	train-rmse:0.086143	test-rmse:0.116645
[305]	train-rmse:0.085965	test-rmse:0.116722
[306]	train-rmse:0.085708	test-rmse:0.116868
[307]	train-rmse:0.08567	test-rmse:0.116727
[308]	train-rmse:0.085536	test-rmse:0.116621
[309]	train-rmse:0.085343	test-rmse:0.116666
[310]	train-rmse:0.085187	test-rmse:0.116578
[311]	train-rmse:0.085045	test-rmse:0.116545
[312]	train-rmse:0.084862	test-rmse:0.116534
[313]	train-rmse:0.084695	test-rmse:0.116448
[314]	train-rmse:0.084562	test-rmse:0.116576
[315]	train-rmse:0.084412	test-rmse:0.116494
[316]	train-rmse:0.084254	test-rmse:0.116542
[317]	train-rmse:0.084128	test-rmse:0.116473
[318]	train-rmse:0.083906	test-rmse:0.116466
[319]	train

[480]	train-rmse:0.068593	test-rmse:0.113862
[481]	train-rmse:0.068508	test-rmse:0.113752
[482]	train-rmse:0.06842	test-rmse:0.113708
[483]	train-rmse:0.068341	test-rmse:0.113755
[484]	train-rmse:0.068293	test-rmse:0.11382
[485]	train-rmse:0.068287	test-rmse:0.113811
[486]	train-rmse:0.068283	test-rmse:0.113757
[487]	train-rmse:0.068215	test-rmse:0.113811
[488]	train-rmse:0.068106	test-rmse:0.11379
[489]	train-rmse:0.068039	test-rmse:0.113742
[490]	train-rmse:0.067906	test-rmse:0.113818
[491]	train-rmse:0.0678	test-rmse:0.113774
[492]	train-rmse:0.067768	test-rmse:0.113821
[493]	train-rmse:0.067738	test-rmse:0.11387
[494]	train-rmse:0.067668	test-rmse:0.113864
[495]	train-rmse:0.067574	test-rmse:0.113783
[496]	train-rmse:0.067513	test-rmse:0.113758
[497]	train-rmse:0.067389	test-rmse:0.113726
Stopping. Best iteration:
[447]	train-rmse:0.070992	test-rmse:0.113075

[0]	train-rmse:11.1849	test-rmse:11.1953
Multiple eval metrics have been passed: 'test-rmse' will be used for early stopping

[167]	train-rmse:0.138822	test-rmse:0.168167
[168]	train-rmse:0.13702	test-rmse:0.166667
[169]	train-rmse:0.135466	test-rmse:0.165626
[170]	train-rmse:0.133657	test-rmse:0.164013
[171]	train-rmse:0.132174	test-rmse:0.162767
[172]	train-rmse:0.130747	test-rmse:0.161785
[173]	train-rmse:0.12954	test-rmse:0.160843
[174]	train-rmse:0.128306	test-rmse:0.159924
[175]	train-rmse:0.12714	test-rmse:0.159085
[176]	train-rmse:0.126027	test-rmse:0.158507
[177]	train-rmse:0.125049	test-rmse:0.157788
[178]	train-rmse:0.123654	test-rmse:0.156816
[179]	train-rmse:0.12274	test-rmse:0.156045
[180]	train-rmse:0.121512	test-rmse:0.155104
[181]	train-rmse:0.120484	test-rmse:0.154205
[182]	train-rmse:0.11934	test-rmse:0.153209
[183]	train-rmse:0.118451	test-rmse:0.152642
[184]	train-rmse:0.117685	test-rmse:0.152062
[185]	train-rmse:0.116703	test-rmse:0.151468
[186]	train-rmse:0.115881	test-rmse:0.151045
[187]	train-rmse:0.115109	test-rmse:0.150339
[188]	train-rmse:0.114447	test-rmse:0.149779
[189]	train-rms

[351]	train-rmse:0.077859	test-rmse:0.130255
[352]	train-rmse:0.077775	test-rmse:0.130272
[353]	train-rmse:0.0777	test-rmse:0.130196
[354]	train-rmse:0.07765	test-rmse:0.130236
[355]	train-rmse:0.077641	test-rmse:0.129976
[356]	train-rmse:0.077582	test-rmse:0.129896
[357]	train-rmse:0.077552	test-rmse:0.129959
[358]	train-rmse:0.077447	test-rmse:0.13015
[359]	train-rmse:0.077277	test-rmse:0.130121
[360]	train-rmse:0.077205	test-rmse:0.130006
[361]	train-rmse:0.077177	test-rmse:0.129941
[362]	train-rmse:0.077159	test-rmse:0.130014
[363]	train-rmse:0.077156	test-rmse:0.130017
[364]	train-rmse:0.07701	test-rmse:0.129876
[365]	train-rmse:0.076915	test-rmse:0.129783
[366]	train-rmse:0.076877	test-rmse:0.129888
[367]	train-rmse:0.076778	test-rmse:0.129781
[368]	train-rmse:0.076743	test-rmse:0.129692
[369]	train-rmse:0.076718	test-rmse:0.129718
[370]	train-rmse:0.076632	test-rmse:0.12967
[371]	train-rmse:0.076471	test-rmse:0.129747
[372]	train-rmse:0.07629	test-rmse:0.129736
[373]	train-rmse:

KeyboardInterrupt: 

In [8]:
submission_col = 'SalePrice'
submission_target = 'test_sub1.csv'
submission_name = 'sample_submission.csv'
submission = pd.DataFrame.from_csv(submission_name)
submission[submission_col] = np.exp(xgclassifier.predict(xg_test))
submission.to_csv(submission_target)